In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

2024-06-03 18:07:44.651805: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-03 18:07:44.651917: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-03 18:07:44.653305: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-03 18:07:44.660457: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 18:07:45.269785: W tensorflow/compiler/tf2

Carga del modelo preentrenado para estimar band gaps de materiales inorgánicos.

In [3]:
modelo = tf.keras.models.load_model('../lr4m5_hse_bg.h5')
modelo.trainable = False
modelo.summary()

2024-06-03 18:07:46.366786: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-03 18:07:46.390103: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-03 18:07:46.390162: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-03 18:07:46.393025: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-03 18:07:46.393078: I external/local_xla/xla/stream_executor

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_layer_000 (InputLaye  [(None, 5120, 1)]            0         []                            
 r)                                                                                               
                                                                                                  
 CONV1D_01 (Conv1D)          (None, 5120, 16)             240       ['input_layer_000[0][0]']     
                                                                                                  
 BN_01 (BatchNormalization)  (None, 5120, 16)             64        ['CONV1D_01[0][0]']           
                                                                                                  
 relu_01 (Activation)        (None, 5120, 16)             0         ['BN_01[0][0]']         

Este es un esquema de la red entrenada. Abrir en una pestaña a parte para agrandar la imagen.

<img src="../lr4m5_hse_bg.png" alt="Red Neuronal" width="300" height="300">

A continuación, se corta el modelo preentrenado para quedarse sólo con la parte convolucional, es decir, la que va desde la capa de entrada hasta la que tiene por nombre "concatenation".

In [4]:
modelCropped = tf.keras.models.Model(inputs = modelo.inputs, outputs = modelo.get_layer('concatenate').output, name = 'ConvNet')

Se cargan los archivos de entrada que corresponden a difractogramas modelados con un tamaño de cristalita de 250 nm y al array que codifica la fracción ocupada de los orbitales de valencia. También se carga el archivo db_peaks.csv, que tiene información del número de reflexiones con intensidad mayor al 5 % en un difractograma modelado con un tamaño de cristalita de 50 nm.

Luego, los difractogramas son escalados a fin de que la intensidad máxima sea 1. Una vez escalados, los archivos de entrada son alimentados para extraer los rasgos que se obtienen hasta la capa "concatenate".

In [5]:
xrd = np.load('../../../hse_GoodDataSet/xset1e4_0250.npy')
elemFeat = np.load('../../../hse_GoodDataset/element_features.npy')
df = pd.read_csv('../../../hse_GoodDataset/dfset_0250.csv')

xrd = xrd/np.max(xrd, axis=(1,2), keepdims=True)

hf = modelCropped.predict([xrd, elemFeat], batch_size=16, verbose=1)

FileNotFoundError: [Errno 2] No such file or directory: '../../../hse_GoodDataSet/xset1e4_0250.npy'

Se guardan los archivos cargados y los descriptores obtenidos de la red preentrenada cortada.

In [6]:
np.save('./hfset.npy', hf)
np.save('./xrdSet_0250.npy',xrd)
np.save('./elem_features.npy', elemFeat)

df.to_csv('./dataset.csv', index=None)

NameError: name 'hf' is not defined

In [22]:
rasgos=pd.read_csv('dataset.csv')
rasgos


,cif,bg_hse
0,SM-1000,3.088441
1,SM-10008,3.409682
2,SM-10012,1.028657
3,SM-10014,3.224964
4,SM-10015,2.711083
...,...,...
8115,SM-9988,3.152018
8116,SM-9989,2.279059
8117,SM-999,2.399116
8118,SM-9991,0.781563


In [19]:
rasgos=np.load('elem_features.npy')
display(pd.DataFrame(rasgos))

ValueError: Must pass 2-d input. shape=(8120, 84, 4)